In [116]:
pip install spotipy

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\1\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [117]:
import csv
import spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth


client_id = 'd061b0cb3ff347acb2968e6585f80104'
client_secret = '287bfbacf27f43f5bfdb325b389a7df5'


client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [118]:
import pandas as pd

playlist_ids = [
    '37i9dQZF1DXcBWIGoYBM5M', 
    '37i9dQZEVXbLiRSasKsNU9', 
    '37i9dQZEVXbMDoHDwVN2tF', 
    '37i9dQZEVXbNG2KDcFcKOF',
    '4E0rXJSUur9qqbn0uZGtwE',
    '77kBUalZyju2tXEuPF36e8',
    '1W2Hms9hAHMBu71mEu0YuV',
    '6CxnmQPhTEpCoB6YtcvKUZ',
    '4y7mJB8JLrjhp2c6k3ADGC'
]

track_data = []

for playlist_id in playlist_ids:
    results = sp.playlist_tracks(playlist_id)
    for item in results['items']:
        track = item['track']
        track_data.append({
            'track_id': track['id'],
            'track_name': track['name'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id'],
            'popularity': track['popularity']
        })

tracks_df = pd.DataFrame(track_data)
print(tracks_df.head())

                 track_id                     track_name        artist_name  \
0  2plbrEY59IikOBgBGLjaoe               Die With A Smile          Lady Gaga   
1  0WbMK4wrZ1wFSty9F7FCgu               Good Luck, Babe!      Chappell Roan   
2  1d7Ptw3qYcfpdLNL5REhtJ                          Taste  Sabrina Carpenter   
3  1Es7AUAhQvapIcoh3qMKDL  Timeless (with Playboi Carti)         The Weeknd   
4  3WOhcATHxK2SLNeP5W3v1v  Guess featuring Billie Eilish         Charli xcx   

                artist_id  popularity  
0  1HY2Jd0NmPuamShAr6KMms         100  
1  7GlBOeep6PqTfFi59PTUUN          96  
2  74KM79TiuVKeVCqs8QtB0B          82  
3  1Xyo4u8uXC1ZmMpatF05PJ          88  
4  25uiPmTg16RbhZWAqwLBy5          91  


In [119]:
audio_features = []
track_ids = tracks_df['track_id'].tolist()

for i in range(0, len(track_ids), 50): 
    features = sp.audio_features(track_ids[i:i+50])
    audio_features.extend(features)

audio_features_df = pd.DataFrame(audio_features)

final_dataset = pd.merge(tracks_df, audio_features_df, left_on='track_id', right_on='id', how='left')

final_dataset.drop(columns=['id'], inplace=True)

print(final_dataset.head())

                 track_id        track_name    artist_name  \
0  2plbrEY59IikOBgBGLjaoe  Die With A Smile      Lady Gaga   
1  2plbrEY59IikOBgBGLjaoe  Die With A Smile      Lady Gaga   
2  2plbrEY59IikOBgBGLjaoe  Die With A Smile      Lady Gaga   
3  2plbrEY59IikOBgBGLjaoe  Die With A Smile      Lady Gaga   
4  0WbMK4wrZ1wFSty9F7FCgu  Good Luck, Babe!  Chappell Roan   

                artist_id  popularity  danceability  energy  key  loudness  \
0  1HY2Jd0NmPuamShAr6KMms         100         0.521   0.592    6    -7.777   
1  1HY2Jd0NmPuamShAr6KMms         100         0.521   0.592    6    -7.777   
2  1HY2Jd0NmPuamShAr6KMms         100         0.521   0.592    6    -7.777   
3  1HY2Jd0NmPuamShAr6KMms         100         0.521   0.592    6    -7.777   
4  7GlBOeep6PqTfFi59PTUUN          96         0.700   0.582   11    -5.960   

   mode  ...  instrumentalness  liveness  valence    tempo            type  \
0     0  ...               0.0    0.1220    0.535  157.969  audio_features   
1 

In [120]:
artist_data = []

artist_ids = tracks_df['artist_id'].unique().tolist()
for i in range(0, len(artist_ids), 50):
    artists = sp.artists(artist_ids[i:i+50])
    for artist in artists['artists']:
        artist_data.append({
            'artist_id': artist['id'],
            'artist_name': artist['name'],
            'genres': artist['genres']
        })

artists_df = pd.DataFrame(artist_data)
final_df = pd.merge(final_dataset, artists_df, on='artist_id', how='left')
print(final_df.head())


                 track_id        track_name  artist_name_x  \
0  2plbrEY59IikOBgBGLjaoe  Die With A Smile      Lady Gaga   
1  2plbrEY59IikOBgBGLjaoe  Die With A Smile      Lady Gaga   
2  2plbrEY59IikOBgBGLjaoe  Die With A Smile      Lady Gaga   
3  2plbrEY59IikOBgBGLjaoe  Die With A Smile      Lady Gaga   
4  0WbMK4wrZ1wFSty9F7FCgu  Good Luck, Babe!  Chappell Roan   

                artist_id  popularity  danceability  energy  key  loudness  \
0  1HY2Jd0NmPuamShAr6KMms         100         0.521   0.592    6    -7.777   
1  1HY2Jd0NmPuamShAr6KMms         100         0.521   0.592    6    -7.777   
2  1HY2Jd0NmPuamShAr6KMms         100         0.521   0.592    6    -7.777   
3  1HY2Jd0NmPuamShAr6KMms         100         0.521   0.592    6    -7.777   
4  7GlBOeep6PqTfFi59PTUUN          96         0.700   0.582   11    -5.960   

   mode  ...  valence    tempo            type  \
0     0  ...    0.535  157.969  audio_features   
1     0  ...    0.535  157.969  audio_features   
2     0 

In [121]:
final_df.to_csv('spotify_music_dataset.csv', index=False)

In [122]:
print(final_df.head())
print(final_df.info())

                 track_id        track_name  artist_name_x  \
0  2plbrEY59IikOBgBGLjaoe  Die With A Smile      Lady Gaga   
1  2plbrEY59IikOBgBGLjaoe  Die With A Smile      Lady Gaga   
2  2plbrEY59IikOBgBGLjaoe  Die With A Smile      Lady Gaga   
3  2plbrEY59IikOBgBGLjaoe  Die With A Smile      Lady Gaga   
4  0WbMK4wrZ1wFSty9F7FCgu  Good Luck, Babe!  Chappell Roan   

                artist_id  popularity  danceability  energy  key  loudness  \
0  1HY2Jd0NmPuamShAr6KMms         100         0.521   0.592    6    -7.777   
1  1HY2Jd0NmPuamShAr6KMms         100         0.521   0.592    6    -7.777   
2  1HY2Jd0NmPuamShAr6KMms         100         0.521   0.592    6    -7.777   
3  1HY2Jd0NmPuamShAr6KMms         100         0.521   0.592    6    -7.777   
4  7GlBOeep6PqTfFi59PTUUN          96         0.700   0.582   11    -5.960   

   mode  ...  valence    tempo            type  \
0     0  ...    0.535  157.969  audio_features   
1     0  ...    0.535  157.969  audio_features   
2     0 

In [123]:
print(final_df.dtypes)

track_id             object
track_name           object
artist_name_x        object
artist_id            object
popularity            int64
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
type                 object
uri                  object
track_href           object
analysis_url         object
duration_ms           int64
time_signature        int64
artist_name_y        object
genres               object
dtype: object


In [124]:
for column in final_df.columns:
    if final_df[column].apply(lambda x: isinstance(x, list)).any():
        print(f"Column '{column}' contains lists.")


Column 'genres' contains lists.


In [125]:
final_df['genres'] = final_df['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

In [126]:
duplicates = final_df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 478


In [128]:
final_df = final_df.drop_duplicates()

duplicates_after = final_df.duplicated().sum()

print(final_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 548 entries, 0 to 1025
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   track_id          548 non-null    object 
 1   track_name        548 non-null    object 
 2   artist_name_x     548 non-null    object 
 3   artist_id         548 non-null    object 
 4   popularity        548 non-null    int64  
 5   danceability      548 non-null    float64
 6   energy            548 non-null    float64
 7   key               548 non-null    int64  
 8   loudness          548 non-null    float64
 9   mode              548 non-null    int64  
 10  speechiness       548 non-null    float64
 11  acousticness      548 non-null    float64
 12  instrumentalness  548 non-null    float64
 13  liveness          548 non-null    float64
 14  valence           548 non-null    float64
 15  tempo             548 non-null    float64
 16  type              548 non-null    object 
 17  u

In [129]:
final_df.head()

,track_id,track_name,artist_name_x,artist_id,popularity,danceability,energy,key,loudness,mode,...,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature,artist_name_y,genres
0,2plbrEY59IikOBgBGLjaoe,Die With A Smile,Lady Gaga,1HY2Jd0NmPuamShAr6KMms,100,0.521,0.592,6,-7.777,0,...,0.535,157.969,audio_features,spotify:track:2plbrEY59IikOBgBGLjaoe,https://api.spotify.com/v1/tracks/2plbrEY59Iik...,https://api.spotify.com/v1/audio-analysis/2plb...,251668,3,Lady Gaga,"art pop, dance pop, pop"
4,0WbMK4wrZ1wFSty9F7FCgu,"Good Luck, Babe!",Chappell Roan,7GlBOeep6PqTfFi59PTUUN,96,0.700,0.582,11,-5.960,0,...,0.785,116.712,audio_features,spotify:track:0WbMK4wrZ1wFSty9F7FCgu,https://api.spotify.com/v1/tracks/0WbMK4wrZ1wF...,https://api.spotify.com/v1/audio-analysis/0WbM...,218424,4,Chappell Roan,indie pop
7,1d7Ptw3qYcfpdLNL5REhtJ,Taste,Sabrina Carpenter,74KM79TiuVKeVCqs8QtB0B,82,0.670,0.910,0,-4.070,0,...,0.786,112.966,audio_features,spotify:track:1d7Ptw3qYcfpdLNL5REhtJ,https://api.spotify.com/v1/tracks/1d7Ptw3qYcfp...,https://api.spotify.com/v1/audio-analysis/1d7P...,157280,4,Sabrina Carpenter,pop
8,1Es7AUAhQvapIcoh3qMKDL,Timeless (with Playboi Carti),The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,88,0.769,0.722,11,-5.485,0,...,0.570,119.973,audio_features,spotify:track:1Es7AUAhQvapIcoh3qMKDL,https://api.spotify.com/v1/tracks/1Es7AUAhQvap...,https://api.spotify.com/v1/audio-analysis/1Es7...,256000,4,The Weeknd,"canadian contemporary r&b, canadian pop, pop"
11,3WOhcATHxK2SLNeP5W3v1v,Guess featuring Billie Eilish,Charli xcx,25uiPmTg16RbhZWAqwLBy5,91,0.776,0.667,7,-6.622,1,...,0.618,130.019,audio_features,spotify:track:3WOhcATHxK2SLNeP5W3v1v,https://api.spotify.com/v1/tracks/3WOhcATHxK2S...,https://api.spotify.com/v1/audio-analysis/3WOh...,145219,4,Charli xcx,"art pop, candy pop, metropopolis, pop, uk pop"


In [130]:
columns_to_drop = ['type', 'uri', 'track_href', 'analysis_url', 'artist_name_y']
final_df = final_df.drop(columns=columns_to_drop)

final_df.head()

,track_id,track_name,artist_name_x,artist_id,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,genres
0,2plbrEY59IikOBgBGLjaoe,Die With A Smile,Lady Gaga,1HY2Jd0NmPuamShAr6KMms,100,0.521,0.592,6,-7.777,0,0.0304,0.3080,0.000000,0.1220,0.535,157.969,251668,3,"art pop, dance pop, pop"
4,0WbMK4wrZ1wFSty9F7FCgu,"Good Luck, Babe!",Chappell Roan,7GlBOeep6PqTfFi59PTUUN,96,0.700,0.582,11,-5.960,0,0.0356,0.0502,0.000000,0.0881,0.785,116.712,218424,4,indie pop
7,1d7Ptw3qYcfpdLNL5REhtJ,Taste,Sabrina Carpenter,74KM79TiuVKeVCqs8QtB0B,82,0.670,0.910,0,-4.070,0,0.0634,0.0939,0.000000,0.3040,0.786,112.966,157280,4,pop
8,1Es7AUAhQvapIcoh3qMKDL,Timeless (with Playboi Carti),The Weeknd,1Xyo4u8uXC1ZmMpatF05PJ,88,0.769,0.722,11,-5.485,0,0.0507,0.0584,0.000003,0.1110,0.570,119.973,256000,4,"canadian contemporary r&b, canadian pop, pop"
11,3WOhcATHxK2SLNeP5W3v1v,Guess featuring Billie Eilish,Charli xcx,25uiPmTg16RbhZWAqwLBy5,91,0.776,0.667,7,-6.622,1,0.0983,0.0146,0.300000,0.0761,0.618,130.019,145219,4,"art pop, candy pop, metropopolis, pop, uk pop"


In [131]:
from sklearn.preprocessing import LabelEncoder

label_encoder_names = LabelEncoder()
final_df['track_name'] = label_encoder_names.fit_transform(final_df['track_name'])

label_encoder_artists = LabelEncoder()
final_df['artist_name_x'] = label_encoder_artists.fit_transform(final_df['artist_name_x'])

print("Data after label encoding artist names:")
print(final_df[['artist_name_x', 'track_name']].head())

Data after label encoding artist names:
    artist_name_x  track_name
0             165          60
4              64          86
7             250         208
8             278         216
11             65          87


In [132]:
genre_counts = final_df['genres'].str.split(', ').explode().value_counts()

top_genres = genre_counts.head(15).index.tolist()

def filter_genres(genres):
    return [genre for genre in genres.split(', ') if genre in top_genres]

final_df['filtered_genres'] = final_df['genres'].apply(filter_genres)

genres_split = final_df['filtered_genres'].str.join(', ').str.get_dummies(sep=', ')

final_df = pd.concat([final_df, genres_split], axis=1)

final_df.drop(['genres', 'filtered_genres'], axis=1, inplace=True)

print(final_df.head())


                  track_id  track_name  artist_name_x               artist_id  \
0   2plbrEY59IikOBgBGLjaoe          60            165  1HY2Jd0NmPuamShAr6KMms   
4   0WbMK4wrZ1wFSty9F7FCgu          86             64  7GlBOeep6PqTfFi59PTUUN   
7   1d7Ptw3qYcfpdLNL5REhtJ         208            250  74KM79TiuVKeVCqs8QtB0B   
8   1Es7AUAhQvapIcoh3qMKDL         216            278  1Xyo4u8uXC1ZmMpatF05PJ   
11  3WOhcATHxK2SLNeP5W3v1v          87             65  25uiPmTg16RbhZWAqwLBy5   

    popularity  danceability  energy  key  loudness  mode  ...  rap  \
0          100         0.521   0.592    6    -7.777     0  ...    0   
4           96         0.700   0.582   11    -5.960     0  ...    0   
7           82         0.670   0.910    0    -4.070     0  ...    0   
8           88         0.769   0.722   11    -5.485     0  ...    0   
11          91         0.776   0.667    7    -6.622     1  ...    0   

    romanian pop  russian alt pop  russian dance  russian edm  \
0              0     

In [133]:
le = LabelEncoder()

final_df['track_id'] = le.fit_transform(final_df['track_id'])
final_df['artist_id'] = le.fit_transform(final_df['artist_id'])


print("DataFrame after Label Encoding:")
print(final_df.head())

DataFrame after Label Encoding:
    track_id  track_name  artist_name_x  artist_id  popularity  danceability  \
0        203          60            165         53         100         0.521   
4         39          86             64        322          96         0.700   
7        118         208            250        309          82         0.670   
8         91         216            278         66          88         0.769   
11       249          87             65         94          91         0.776   

    energy  key  loudness  mode  ...  rap  romanian pop  russian alt pop  \
0    0.592    6    -7.777     0  ...    0             0                0   
4    0.582   11    -5.960     0  ...    0             0                0   
7    0.910    0    -4.070     0  ...    0             0                0   
8    0.722   11    -5.485     0  ...    0             0                0   
11   0.667    7    -6.622     1  ...    0             0                0   

    russian dance  russian edm

In [134]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numerical_columns = ['popularity', 'danceability', 'energy', 'key', 
                     'loudness', 'mode', 'speechiness', 
                     'acousticness', 'instrumentalness', 
                     'liveness', 'valence', 'tempo', 
                     'duration_ms', 'time_signature']

final_df[numerical_columns] = scaler.fit_transform(final_df[numerical_columns])
final_df.head()

,track_id,track_name,artist_name_x,artist_id,popularity,danceability,energy,key,loudness,mode,...,rap,romanian pop,russian alt pop,russian dance,russian edm,russian gangster rap,russian hip hop,russian pop,russian trap,singer-songwriter pop
0,203,60,165,53,1.955076,-1.536365,-0.608700,0.087248,-0.706410,-0.670623,...,0,0,0,0,0,0,0,0,0,0
4,39,86,64,322,1.803214,-0.051571,-0.676757,1.477122,0.118599,-0.670623,...,0,0,0,0,0,0,0,0,0,0
7,118,208,250,309,1.271700,-0.300419,1.555511,-1.580601,0.976753,-0.670623,...,0,0,0,0,0,0,0,0,0,0
8,91,216,278,66,1.499492,0.520780,0.276040,1.477122,0.334273,-0.670623,...,0,0,0,0,0,0,0,0,0,0
11,249,87,65,94,1.613388,0.578844,-0.098273,0.365222,-0.181982,1.491150,...,0,0,0,0,0,0,0,0,0,0
